In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from functions import *
from helpers import *
from proj1_helpers import *
from costs import *
from data_preprocessing import *

%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
# TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
print("loading of the data : done")

loading of the data : done


## Data preprocessing

In [11]:
# reduction of the size of the training data set: 
# random selection of the data
num_samples=10000
seed=3
y, tX = select_random(y, tX, num_samples, seed)
print ("random selection of samples : done")
print( "number of samples : ", y.shape[0])

##### Splitting the data
xtrain1,xvalid1,ytrain,yvalid=split_data(tX, y, 0.5)
print( "splitting of the data : done ")
print("number of training samples :", xtrain1.shape[0])

#### Data preprocessing : perform a pca on xtrain
ratio_pca=0.95

# finding the projector
U, k=my_pca(xtrain1, ratio_pca)
print("shape of the projector : ", U.shape)

# projecting the data 
xtrain2=np.dot(xtrain1,U)
xvalid2=np.dot(xvalid1,U)
print(" shape of the reduced training set : ", xtrain2.shape)
print(" shape of the reduced validation set : ", xvalid2.shape )

#then adding a one before each sample (to enable a constant in linear regression)
xtrain=np.ones((xtrain2.shape[0],xtrain2.shape[1]+1))
xvalid=np.ones((xvalid2.shape[0],xvalid2.shape[1]+1))
xtrain[:,1:]=xtrain2
xvalid[:,1:]=xvalid2
print(" shape of the final training set : ", xtrain.shape)
print(" shape of the final validation set : ", xvalid.shape )

random selection of samples : done
number of samples :  10000
splitting of the data : done 
number of training samples : 5000
eigenvalues sorted by decreasing order :  [  9.97158834e+09   2.31050426e+09   7.74488545e+08   1.68468587e+08
   8.08644919e+07   1.06800004e+07   9.24903150e+06   5.33883911e+06
   4.68048901e+06   2.43262870e+06   1.87726697e+06   1.68157450e+06
   8.48220325e+05   6.51655403e+05   1.95214150e+05   1.98991224e+04
   1.73455149e+04   1.31601769e+04   1.22306220e+04   8.22678440e+03
   6.08754106e+03   4.71302669e+03   3.62901398e+03   3.27812127e+03
   3.09709202e+03   9.58445704e+02   5.40310786e+02   2.74509240e+02
   1.24435116e+02   3.67228645e-04]
 debugging : check the values :
k :  3
partial sum of eigenvalues :  13056581143.7
total sum of eigenvalues :  13343642707.0
ratio :  0.978487016652
shape of the projector :  (30, 3)
 shape of the reduced training set :  (5000, 3)
 shape of the reduced validation set :  (5000, 3)
 shape of the final training set

## Machine Learning : test of the basic functions

In [12]:
### Training of the model : test of least_squares_GD
#weights = train_data(xtrain,ytrain,n_regression=1,gamma=0.0000001,max_iters=500)

### Training of the model : test of least_squares (analytical)
#weights = train_data(xtrain, ytrain, n_regression=3)

### Training of the model : test of least_squares_SGD
#weights = train_data(xtrain, ytrain, n_regression=2, gamma=10e-13, max_iters=500)

### Training of the model : test of ridge_regression
#weights = train_data(xtrain, ytrain, n_regression=4,lambd=100)

print("Training of the model : done")

Training of the model : done


In [13]:
### Measure of the MSE for the trained model, over the validation set  : 

#MSE=compute_loss(yvalid,xvalid,weights) 

#print ( "MSE computed, value : ", MSE)

## Machine Learning :  Comparison of models 

In [14]:
# First comparison : for the given training and validation set : choice of the best lambd_ for the ridge regression
lambdas = np.logspace(-3, 3, 500)  
#setting references
w_ref = 0
lambd_ref=0
loss_ref = float("inf")
for lambd_ in lambdas:
    #training the model for the ridge regression given lambda
    weights = train_data(xtrain, ytrain, n_regression=4,lambd=lambd_)
    #computing its score
    mse = compute_loss(yvalid, xvalid, weights)
    #if its cost is better than the previous better one, i retain this cost and the associated weight, lambd
    if mse<loss_ref:
        w_ref=weights
        loss_ref=mse
        lambd_ref=lambd_
    print("study of the parameter :", str(lambd_), " done. Cost associated : ", str(mse) )
weights=w_ref
loss=loss_ref
print("comparison of the ridge regression parameters : done")
print("best parameter : " , str(lambd_ref))
print(" associated MSE cost : " , str(loss))
print("associated weight vector" , str(weights))

study of the parameter : 0.001  done. Cost associated :  0.43080603256
study of the parameter : 0.00102807322383  done. Cost associated :  0.430803356653
study of the parameter : 0.00105693455356  done. Cost associated :  0.430800606262
study of the parameter : 0.00108660611385  done. Cost associated :  0.430797779333
study of the parameter : 0.0011171106505  done. Cost associated :  0.430794873756
study of the parameter : 0.00114847154784  done. Cost associated :  0.430791887362
study of the parameter : 0.00118071284667  done. Cost associated :  0.430788817927
study of the parameter : 0.00121385926269  done. Cost associated :  0.430785663162
study of the parameter : 0.00124793620547  done. Cost associated :  0.430782420721
study of the parameter : 0.00128296979789  done. Cost associated :  0.430779088193
study of the parameter : 0.0013189868962  done. Cost associated :  0.430775663102
study of the parameter : 0.00135601511057  done. Cost associated :  0.430772142904
study of the param

## Generate predictions and save ouput in csv format for submission:

In [83]:
DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [84]:
OUTPUT_PATH = '../results/result2.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)